In [1]:
from operator import add
import json
import shapely.geometry as geom

with open('block-groups-polygons-simple.geojson', 'r') as f1:
    js_blocks = json.load(f1)
        
with open('station.json', 'r') as f2:
    js_stations = json.load(f2)

def indexZones(shapeFilename):
    import rtree
    import fiona.crs
    import geopandas as gpd
    index = rtree.Rtree()
    zones = gpd.read_file(shapeFilename).to_crs(fiona.crs.from_epsg(2263))
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

# for station in js_stations['stationBeanList']:
#     lat = station['latitude']
#     lon = station['longitude']
#     stationLocation = geom.Point(lat,lon)
#     for feature in js_blocks['features']:
#         polygon = geom.shape(feature['geometry'])
#         if polygon.contains(stationLocation):
#             print (station[stationName], feature['properties'])

def findBlock(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if any(map(lambda x: x.contains(p), zones.geometry[idx])):
            return zones['properties'][idx]
    return -1

index, zones = indexZones('block-groups-polygons-simple.geojson.geojson')

lat = -73.82266 
lon = 40.703585
stationLocation = geom.Point(lat,lon)
for feature in js_blocks['features']:
    polygon = geom.shape(feature['geometry'])
    findBlock(stationLocation, )
        

      



ImportError: No module named fiona.crs

In [2]:
import fiona

ImportError: No module named fiona